In [1]:
import os 
import torch
print(torch.__version__)
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # For synchronous execution of CPU and GPU
client_device = torch.device("cuda:2")
server_device = torch.device("cuda:1")
model_device = torch.device("cuda:0")

1.13.1+cu116


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoConfig
from pipe_mixformer_sequential import ClientSideMixFormerSequentialForCausalLM, ServerSideMixFormerSequentialForCausalLM
from configuration_mixformer_sequential import MixFormerSequentialConfig
from modeling_mixformer_sequential import MixFormerSequentialForCausalLM
config = AutoConfig.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
# config = MixFormerSequentialConfig(n_layer=25)
client_model = ClientSideMixFormerSequentialForCausalLM(config)
server_model = ServerSideMixFormerSequentialForCausalLM(config)
model = MixFormerSequentialForCausalLM(config)

[2023-10-26 09:30:09,847] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
MixFormerSequentialConfig {
  "_name_or_path": "microsoft/phi-1_5",
  "activation_function": "gelu_new",
  "architectures": [
    "MixFormerSequentialForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "microsoft/phi-1_5--configuration_mixformer_sequential.MixFormerSequentialConfig",
    "AutoModelForCausalLM": "microsoft/phi-1_5--modeling_mixformer_sequential.MixFormerSequentialForCausalLM"
  },
  "embd_pdrop": 0.0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "mixformer-sequential",
  "n_embd": 2048,
  "n_head": 32,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 2048,
  "resid_pdrop": 0.0,
  "rotary_dim": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.29.2",
  "vocab_size": 51200
}

MixFormerSequentialConfig {
  "_name_or_path": "microsoft/phi-1_5",
  "activation_function": "gelu_new"

In [3]:

from transformers import AutoTokenizer, AutoModelForCausalLM
from pipe_mixformer_sequential import ClientSideMixFormerSequentialForCausalLM, ServerSideMixFormerSequentialForCausalLM
from peft import LoraConfig, get_peft_model
import torch

model_name = ("gpt2", "microsoft/phi-1_5")
tokenizer = AutoTokenizer.from_pretrained(
    model_name[1], 
    trust_remote_code=True,
    cache_dir="/app/.huggingface_cache/model/"
)
model = AutoModelForCausalLM.from_pretrained(
    model_name[1],
    cache_dir="/app/.huggingface_cache/model/",
    trust_remote_code=True,
    torch_dtype=torch.float16
)
client_model = ClientSideMixFormerSequentialForCausalLM(model.config)
server_model = ServerSideMixFormerSequentialForCausalLM(model.config)


MixFormerSequentialConfig {
  "_name_or_path": "microsoft/phi-1_5",
  "activation_function": "gelu_new",
  "architectures": [
    "MixFormerSequentialForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "microsoft/phi-1_5--configuration_mixformer_sequential.MixFormerSequentialConfig",
    "AutoModelForCausalLM": "microsoft/phi-1_5--modeling_mixformer_sequential.MixFormerSequentialForCausalLM"
  },
  "embd_pdrop": 0.0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "mixformer-sequential",
  "n_embd": 2048,
  "n_head": 32,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 2048,
  "resid_pdrop": 0.0,
  "rotary_dim": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.29.2",
  "vocab_size": 51200
}

MixFormerSequentialConfig {
  "_name_or_path": "microsoft/phi-1_5",
  "activation_function": "gelu_new",
  "architectures": [
    "MixFormerSequentialForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "microsoft/phi-1_5--c

In [4]:
import numpy as np

def get_fp32_layers(model):
    fp32_layers=[]
    for k, v in model.state_dict().items():
        print(f"{k}: {v.dtype}")
        if(v.dtype == torch.float32):
            fp32_layers.append(k)
    return fp32_layers


def load_pretrained_weights(sub_model, model, fp32_layers):    
    np_param={}
    for module_name, param in model.state_dict().items():
        np_param[module_name] = param.numpy().astype(np.float16)
        
    # sub_model = sub_model.to(torch.float16)
    for module_name, param in sub_model.state_dict().items():
        if(model_name not in fp32_layers):
            sub_model.state_dict()[module_name] = sub_model.state_dict()[module_name].to(torch.float16)
        
        sub_model.state_dict()[module_name].data.copy_(torch.from_numpy(np_param[module_name]))
        sub_model.state_dict()[module_name]


In [5]:

fp32_layers = get_fp32_layers(model)
load_pretrained_weights(client_model, model, fp32_layers)
load_pretrained_weights(server_model, model, fp32_layers)

layers.0.wte.weight: torch.float16
layers.1.ln.weight: torch.float16
layers.1.ln.bias: torch.float16
layers.1.mixer.rotary_emb.inv_freq: torch.float32
layers.1.mixer.Wqkv.weight: torch.float16
layers.1.mixer.Wqkv.bias: torch.float16
layers.1.mixer.out_proj.weight: torch.float16
layers.1.mixer.out_proj.bias: torch.float16
layers.1.mlp.fc1.weight: torch.float16
layers.1.mlp.fc1.bias: torch.float16
layers.1.mlp.fc2.weight: torch.float16
layers.1.mlp.fc2.bias: torch.float16
layers.2.ln.weight: torch.float16
layers.2.ln.bias: torch.float16
layers.2.mixer.rotary_emb.inv_freq: torch.float32
layers.2.mixer.Wqkv.weight: torch.float16
layers.2.mixer.Wqkv.bias: torch.float16
layers.2.mixer.out_proj.weight: torch.float16
layers.2.mixer.out_proj.bias: torch.float16
layers.2.mlp.fc1.weight: torch.float16
layers.2.mlp.fc1.bias: torch.float16
layers.2.mlp.fc2.weight: torch.float16
layers.2.mlp.fc2.bias: torch.float16
layers.3.ln.weight: torch.float16
layers.3.ln.bias: torch.float16
layers.3.mixer.rota

In [6]:
for k, v in client_model.state_dict().items():
    print(f"{k}: {v.dtype}")
    

layers.0.wte.weight: torch.float32
layers.1.ln.weight: torch.float32
layers.1.ln.bias: torch.float32
layers.1.mixer.rotary_emb.inv_freq: torch.float32
layers.1.mixer.Wqkv.weight: torch.float32
layers.1.mixer.Wqkv.bias: torch.float32
layers.1.mixer.out_proj.weight: torch.float32
layers.1.mixer.out_proj.bias: torch.float32
layers.1.mlp.fc1.weight: torch.float32
layers.1.mlp.fc1.bias: torch.float32
layers.1.mlp.fc2.weight: torch.float32
layers.1.mlp.fc2.bias: torch.float32
layers.2.ln.weight: torch.float32
layers.2.ln.bias: torch.float32
layers.2.mixer.rotary_emb.inv_freq: torch.float32
layers.2.mixer.Wqkv.weight: torch.float32
layers.2.mixer.Wqkv.bias: torch.float32
layers.2.mixer.out_proj.weight: torch.float32
layers.2.mixer.out_proj.bias: torch.float32
layers.2.mlp.fc1.weight: torch.float32
layers.2.mlp.fc1.bias: torch.float32
layers.2.mlp.fc2.weight: torch.float32
layers.2.mlp.fc2.bias: torch.float32
layers.3.ln.weight: torch.float32
layers.3.ln.bias: torch.float32
layers.3.mixer.rota

In [7]:
for k, v in server_model.state_dict().items():
    print(f"{k}: {v.dtype}")
    

layers.11.ln.weight: torch.float32
layers.11.ln.bias: torch.float32
layers.11.mixer.rotary_emb.inv_freq: torch.float32
layers.11.mixer.Wqkv.weight: torch.float32
layers.11.mixer.Wqkv.bias: torch.float32
layers.11.mixer.out_proj.weight: torch.float32
layers.11.mixer.out_proj.bias: torch.float32
layers.11.mlp.fc1.weight: torch.float32
layers.11.mlp.fc1.bias: torch.float32
layers.11.mlp.fc2.weight: torch.float32
layers.11.mlp.fc2.bias: torch.float32
layers.12.ln.weight: torch.float32
layers.12.ln.bias: torch.float32
layers.12.mixer.rotary_emb.inv_freq: torch.float32
layers.12.mixer.Wqkv.weight: torch.float32
layers.12.mixer.Wqkv.bias: torch.float32
layers.12.mixer.out_proj.weight: torch.float32
layers.12.mixer.out_proj.bias: torch.float32
layers.12.mlp.fc1.weight: torch.float32
layers.12.mlp.fc1.bias: torch.float32
layers.12.mlp.fc2.weight: torch.float32
layers.12.mlp.fc2.bias: torch.float32
layers.13.ln.weight: torch.float32
layers.13.ln.bias: torch.float32
layers.13.mixer.rotary_emb.inv

In [8]:
# for k, v in client_model.state_dict().items():
#     torch.set_default_dtype(torch.float16)
#     print(f"{k}: {v.dtype}")
    
# for k, v in client_model.state_dict().items():    
#     print(f"{k}: {v.dtype}")

In [9]:
client_model = client_model.to(client_device)
server_model = server_model.to(server_device)

In [10]:
for k, v in client_model.state_dict().items():
    print(f"{k}: {v.dtype}")
    

layers.0.wte.weight: torch.float32
layers.1.ln.weight: torch.float32
layers.1.ln.bias: torch.float32
layers.1.mixer.rotary_emb.inv_freq: torch.float32
layers.1.mixer.Wqkv.weight: torch.float32
layers.1.mixer.Wqkv.bias: torch.float32
layers.1.mixer.out_proj.weight: torch.float32
layers.1.mixer.out_proj.bias: torch.float32
layers.1.mlp.fc1.weight: torch.float32
layers.1.mlp.fc1.bias: torch.float32
layers.1.mlp.fc2.weight: torch.float32
layers.1.mlp.fc2.bias: torch.float32
layers.2.ln.weight: torch.float32
layers.2.ln.bias: torch.float32
layers.2.mixer.rotary_emb.inv_freq: torch.float32
layers.2.mixer.Wqkv.weight: torch.float32
layers.2.mixer.Wqkv.bias: torch.float32
layers.2.mixer.out_proj.weight: torch.float32
layers.2.mixer.out_proj.bias: torch.float32
layers.2.mlp.fc1.weight: torch.float32
layers.2.mlp.fc1.bias: torch.float32
layers.2.mlp.fc2.weight: torch.float32
layers.2.mlp.fc2.bias: torch.float32
layers.3.ln.weight: torch.float32
layers.3.ln.bias: torch.float32
layers.3.mixer.rota

In [11]:
for k, v in server_model.state_dict().items():
    print(f"{k}: {v.dtype}")

layers.11.ln.weight: torch.float32
layers.11.ln.bias: torch.float32
layers.11.mixer.rotary_emb.inv_freq: torch.float32
layers.11.mixer.Wqkv.weight: torch.float32
layers.11.mixer.Wqkv.bias: torch.float32
layers.11.mixer.out_proj.weight: torch.float32
layers.11.mixer.out_proj.bias: torch.float32
layers.11.mlp.fc1.weight: torch.float32
layers.11.mlp.fc1.bias: torch.float32
layers.11.mlp.fc2.weight: torch.float32
layers.11.mlp.fc2.bias: torch.float32
layers.12.ln.weight: torch.float32
layers.12.ln.bias: torch.float32
layers.12.mixer.rotary_emb.inv_freq: torch.float32
layers.12.mixer.Wqkv.weight: torch.float32
layers.12.mixer.Wqkv.bias: torch.float32
layers.12.mixer.out_proj.weight: torch.float32
layers.12.mixer.out_proj.bias: torch.float32
layers.12.mlp.fc1.weight: torch.float32
layers.12.mlp.fc1.bias: torch.float32
layers.12.mlp.fc2.weight: torch.float32
layers.12.mlp.fc2.bias: torch.float32
layers.13.ln.weight: torch.float32
layers.13.ln.bias: torch.float32
layers.13.mixer.rotary_emb.inv

In [12]:
from peft import LoraConfig, get_peft_model

lora_modules=["Wqkv"] 
lora_config = LoraConfig(
    r=1,  # dimension of the updated matrices
    lora_alpha=64,  # parameter for scaling
    target_modules=lora_modules,
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
client_model = get_peft_model(client_model, lora_config)
server_model = get_peft_model(server_model, lora_config)

In [13]:
from datasets import load_dataset
from transformers import default_data_collator
from data_utils import transform_data_to_fedml_format, group_texts, tokenize_function
from functools import partial
from torch.utils.data import DataLoader
import torch

# resize embeddings
embedding_size = client_model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    client_model.resize_token_embeddings(len(tokenizer))

# load data

block_size = tokenizer.model_max_length
raw_datasets = load_dataset(
    "wikitext",
    "wikitext-2-raw-v1",    
    cache_dir="/app/.huggingface_cache/dataset/",
    streaming=False
)
column_names = list(raw_datasets["train"].features)

# data preprocessing 
__tokenize_function = partial(tokenize_function, text_column_name="text", tokenizer=tokenizer)
tokenized_datasets = raw_datasets.map(
                    __tokenize_function,
                    batched=True,
                    remove_columns=column_names,
                    desc="Running tokenizer on dataset",
                )

__group_texts = partial(group_texts, block_size=block_size)
lm_datasets = tokenized_datasets.map(
                __group_texts,
                batched=True,
                # num_proc=1,
                # load_from_cache_file=not data_args.overwrite_cache,
                # desc=f"Grouping texts in chunks of {block_size}",
            )
lm_datasets['train'].set_format("torch", device=client_device)
train_dataloader = DataLoader(lm_datasets['train'], shuffle=True, collate_fn=default_data_collator, batch_size=1)
print(lm_datasets['train'])
print(train_dataloader)
no_decay = ["bias", "layer_norm.weight"]
client_grouped_parameters = [
    {
        "params": [p for n, p in client_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in client_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
server_grouped_parameters = [
    {
        "params": [p for n, p in server_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in server_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
model_grouped_paramters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
server_optimizer = torch.optim.AdamW(server_grouped_parameters, lr=0.001)
client_optimizer = torch.optim.AdamW(client_grouped_parameters, lr=0.001)
model_optimizer = torch.optim.AdamW(model_grouped_paramters, lr=0.001)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1150
})


In [14]:
def unfreeze_params(model):
    for param in model.parameters():
        param.requires_grad = True
        
def print_trainable_params(model):
    for name, param in model.named_parameters():
        if(param.requires_grad):
            print(name, param.requires_grad)
        
def print_grad(model):
    for name, param in model.named_parameters():
        if(param.requires_grad):
            print(name, param.grad)
            
# unfreeze_params(client_model)
# unfreeze_params(server_model)
print("\n\n---- trainable params of server_model ----")
print_trainable_params(server_model)
print("\n\n---- trainable params of client_model ----")
print_trainable_params(client_model)
print(f"\n {len(train_dataloader)}")



---- trainable params of server_model ----
base_model.model.layers.11.mixer.Wqkv.lora_A.default.weight True
base_model.model.layers.11.mixer.Wqkv.lora_B.default.weight True
base_model.model.layers.12.mixer.Wqkv.lora_A.default.weight True
base_model.model.layers.12.mixer.Wqkv.lora_B.default.weight True
base_model.model.layers.13.mixer.Wqkv.lora_A.default.weight True
base_model.model.layers.13.mixer.Wqkv.lora_B.default.weight True
base_model.model.layers.14.mixer.Wqkv.lora_A.default.weight True
base_model.model.layers.14.mixer.Wqkv.lora_B.default.weight True
base_model.model.layers.15.mixer.Wqkv.lora_A.default.weight True
base_model.model.layers.15.mixer.Wqkv.lora_B.default.weight True
base_model.model.layers.16.mixer.Wqkv.lora_A.default.weight True
base_model.model.layers.16.mixer.Wqkv.lora_B.default.weight True
base_model.model.layers.17.mixer.Wqkv.lora_A.default.weight True
base_model.model.layers.17.mixer.Wqkv.lora_B.default.weight True
base_model.model.layers.18.mixer.Wqkv.lora_A.

In [15]:
import math
import time
import numpy as np

print_grad = False
model.train()
client_model.train()
server_model.train()
print(len(train_dataloader))

latency = {
    "client": [],
    "server": [],
    "end-to-end": [],
    "model": []
}


scaler = torch.cuda.amp.GradScaler()
for step, batch in enumerate(train_dataloader):
    print(f"step = {step} ")
    
    for key in batch.keys():
        batch[key] = batch[key].to(device=client_device)
    
    client_model, server_model = client_model.to(device=client_device), server_model.to(device=server_device)

    # print(client_model.dtype)
    # print(server_model.dtype)

    client_start_time = time.perf_counter()
    input_ids, past_key_values, attention_mask, labels, acts = client_model(**batch)
    client_end_time = time.perf_counter()
    
    acts.retain_grad()
    input_ids, attention_mask, labels, acts = input_ids.to(device=server_device), attention_mask.to(device=server_device), labels.to(device=server_device), acts.to(device=server_device)
    
    server_start_time = time.perf_counter()
    split_outputs = server_model(input_ids=input_ids, past_key_values=past_key_values, attention_mask=attention_mask, labels=labels, hidden_layer_input=acts)
    server_end_time = time.perf_counter()
        
    split_loss = split_outputs.loss
    split_loss.backward()
    split_perplexity = math.exp(split_loss)
    
    server_optimizer.step()
    client_optimizer.step()
    
    # scaler.update()
    if(print_grad):
        print("\n\n---- grad on server_model ----")
        print_grad(server_model)    
        print("\n\n---- grad on client_model ----")
        print_grad(client_model)
    
    latency["client"].append(client_end_time-client_start_time)
    latency["server"].append(server_end_time-server_start_time)
    latency["end-to-end"].append(server_end_time-client_start_time)
    print(f"  - split_loss = {split_loss}")
    print(f"  - split_perplexity = {split_perplexity}")
    print(f"  - forward latency (sec): Client = {np.average(latency['client'])}  ||  Server = {np.average(latency['server'])}  || End-to-end = {np.average(latency['end-to-end'])}")
    
    server_optimizer.zero_grad()
    client_optimizer.zero_grad()
    
    # # Non-split training
    # model = model.to(model_device)
    # for key in batch.keys():
    #     batch[key] = batch[key].to(model_device)
    # model_start_time = time.perf_counter()
    # model_output = model(**batch)
    # model_end_time = time.perf_counter()
    # model_loss = model_output.loss
    
    # model_perplexity = math.exp(model_loss)
    # model_loss.backward()
    # model_optimizer.step()
    # model_optimizer.zero_grad()
    # latency['model'].append(model_end_time-model_start_time)
    # print(f"  - model_loss = {model_loss}")
    # print(f"  - model_perplexity = {model_perplexity}")
    # print(f"  - Latency (sec): model = {np.average(latency['model'])}\n")


1150
step = 0 


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 2; 10.76 GiB total capacity; 9.39 GiB already allocated; 429.56 MiB free; 9.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF